### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [596]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store as data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()
#purchase_data.describe()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [492]:
print("Total number of players: "+ str(len(purchase_data['SN'].unique().tolist())))


Total number of players: 576


## Purchasing Analysis (Total)
 
- Run basic calculations to obtain number of unique items, average price, etc.
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [551]:
# Run basic calculations to obtain number of unique items, average price, etc.
unique_item=len(purchase_data['Item ID'].unique().tolist())
ave_price=purchase_data['Price'].mean()
num_purchases=purchase_data['Purchase ID'].count()
tot_revenue=purchase_data['Price'].sum()

table = [('Number of Unique Items', [unique_item]), ('Average Price', [ave_price]), 
         ('Number of Purchases', [num_purchases]),('Total Revenue', [tot_revenue])]

#Create a summary data frame to hold the results
df_table=pd.DataFrame.from_items(table)

#Optional: give the displayed data cleaner formatting
df_table["Number of Unique Items"] = df_table["Number of Unique Items"].astype(float).map("{:,.0f}".format)
df_table["Average Price"] = df_table["Average Price"].astype(float).map("${:,.2f}".format)
df_table["Total Revenue"] = df_table["Total Revenue"].astype(float).map("${:,.2f}".format)
# Display the summary data frame
df_table


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  # This is added back by InteractiveShellApp.init_path()


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics
- Percentage and Count of Male Players
- Percentage and Count of Female Players
- Percentage and Count of Other / Non-Disclosed

In [564]:
# Create a counter variable
purchase_data["counter"]= 1
purchase_data["counter"].count()

# Group by gender and drop duplicates
gender_count=purchase_data.drop_duplicates(subset ="SN", inplace = True, keep='first')

gender_count = purchase_data.groupby("Gender")
gender_count.count()
# Stats by gender 
gender_count = gender_count[["counter"]].sum()
gender_per=gender_count.apply(lambda x: 100*x/x.sum()).reset_index()

#Merge dataframes
gender_stats = pd.merge(gender_per, gender_count,how='outer', on="Gender")

# Renaming and formatting
gender_stats_ren = gender_stats.rename(columns = {"counter_x": "Percentage of Players", "counter_y":"Total Count"}) 
gender_stats_ren["Percentage of Players"] = gender_stats_ren["Percentage of Players"].astype(float).map("{:,.2f}".format)

gender_stats_ren.head()


,Gender,Percentage of Players,Total Count
0,Female,14.06,81
1,Male,84.03,484
2,Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [252]:
# Average total purchase per person
per_pur= purchase_data.groupby(["Gender", "SN"])
per_pur.count()
per_pur_ave = per_pur[["Price"]].sum()
per_pur_ave = per_pur_ave.groupby("Gender")
per_pur_ave = per_pur_ave.mean()
per_pur_ave.head()

# Average price per gender
gender_pur = purchase_data.groupby("Gender")
gender_per_pur = purchase_data.groupby(["Gender", "SN"])
gender_avprice = gender_pur[["Price"]].mean()
gender_avprice.head()
gender_pur_stats = pd.merge(per_pur_ave, gender_avprice, how='outer', on="Gender")
gender_pur_stats.head()

# Total purchase per gender
gender_pur = gender_pur[["Price"]].sum()
gender_pur.head()
gender_pur_stats = pd.merge(gender_pur_stats, gender_pur, how='outer', on="Gender")
gender_pur_stats.head()

# Add total number of purchases by gender
gender_pur_stats = pd.merge(gender_pur_stats, gender_count, how='outer', on="Gender")
gender_pur_stats.head()


# Rename columns
gender_pur_stats = gender_pur_stats.rename(columns = {"Price_x":  "Avg Total Purchase per Person", 
                                                      "Price_y": "Average Price",
                                                     "Price": "Total Purchase Value",
                                                     "counter": "Purchase Count"}) 

# Formatting
gender_pur_stats["Avg Total Purchase per Person"] = gender_pur_stats["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)
gender_pur_stats["Average Price"] = gender_pur_stats["Average Price"].astype(float).map("${:,.2f}".format)
gender_pur_stats["Total Purchase Value"] = gender_pur_stats["Total Purchase Value"].astype(float).map("${:,.2f}".format)

gender_pur_stats.head()


,Avg Total Purchase per Person,Average Price,Total Purchase Value,Purchase Count
Gender,,,,
Female,$4.47,$3.20,$361.94,113
Male,$4.07,$3.02,"$1,967.64",652
Other / Non-Disclosed,$4.56,$3.35,$50.19,15


## Age Demographics

* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table

In [648]:
pur_data=purchase_data.drop_duplicates(subset ="SN", keep='first', inplace = True)
purchase_data.count()
pur_data = purchase_data[["Age"]]
pur_data.head()
pur_data.count()

# Creating bins and slicing data
bin_ages=[0,9, 14, 19, 24, 29, 34, 39, 50]
label_ages = ["<=10", "11-15", "15-20", "20-24", "25-29", "30-34", "35-39", "40+"]
pur_data["Age Group"] = pd.cut(pur_data["Age"], bin_ages, labels=label_ages)
pur_data.head()

# # Calculate count of people by age
pur_data["counter"]= 1
pur_data["counter"].count()
age_count = pur_data.groupby("Age Group")
age_count.head()

age_count = age_count[["counter"]].sum()
age_count.head(11)

# # Percentage of people by age
age_per=age_count.apply(lambda x: 100*x/x.sum()).reset_index()
age_per.head(11)

# # Create a table with summary stats by age
age_stats = pd.merge(age_per, age_count, how='outer', on="Age Group")
age_stats.head(11)

#Rename columns
age_stats = age_stats.rename(columns = {"counter_x":  "Percentage of Players", "counter_y": "Total Count"}) 

# Formatting
age_stats["Percentage of Players"] = age_stats["Percentage of Players"].astype(float).map("{:,.2f}".format)

age_stats.head(11)



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Age Group,Percentage of Players,Total Count
0,<=10,2.95,17
1,11-15,3.82,22
2,15-20,18.58,107
3,20-24,44.79,258
4,25-29,13.37,77
5,30-34,9.03,52
6,35-39,5.38,31
7,40+,2.08,12


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [651]:
# Bin the purchase_data frame by age
purchase_data2 = pd.read_csv(file_to_load)
purchase_data2.count()

purchase_data2["Age Group"] = pd.cut(purchase_data["Age"], bin_ages)
purchase_data.count()
# Count
purchase_data2["counter"]= 1
pur_age_count = purchase_data2.groupby("Age Group")
pur_age_count = pur_age_count[["counter"]].count()

# Average price
pur_age_price = purchase_data2.groupby("Age Group")
pur_age_price = pur_age_price[["Price"]].mean()
age_pur_stats = pd.merge(pur_age_count, pur_age_price,how='outer', on="Age Group")
age_pur_stats = age_pur_stats.rename(columns = {"counter":  "Purchase Count", 
                                                "Price": "Average Price"}) 
age_pur_stats.head(11)

# # Average purchase value per groups of age
# pur_age_value = purchase_data.groupby("Age Group")
# pur_age_value = pur_age_value[["Price"]].sum()
# age_pur_stats = pd.merge(age_pur_stats, pur_age_value,how='outer', on="Age Group")
# age_pur_stats = age_pur_stats.rename(columns = {"Price":  "Total Purchase Value"}) 
# age_pur_stats.head(11)

# # # Average purchase per person
# age_pur_ave_per= purchase_data2.groupby(["SN", "Age Group"])
# age_pur_ave_per.count()
# age_pur_ave_per = age_pur_ave_per[["Price"]].sum()
# age_pur_ave = age_pur_ave_per.groupby("Age Group")
# age_pur_ave = age_pur_ave.mean()
# age_pur_ave.head(8)

,Purchase Count,Average Price
Age Group,,
"(0, 9]",17,3.390000
"(9, 14]",22,3.074545
"(14, 19]",107,3.101682
"(19, 24]",258,3.063527
"(24, 29]",77,2.908182
"(29, 34]",52,2.921538
"(34, 39]",31,3.510000
"(39, 50]",12,3.037500


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [452]:
# Group data by person
pur_person= purchase_data.groupby(["SN"])

# Price
pur_person_ave=pur_person[["Price"]].mean()
pur_person_ave["Price"] = pur_person_ave["Price"].astype(float).map("${:,.2f}".format)
pur_person_ave = pur_person_ave.rename(columns = {"Price": "Average Purchase Price"}) 

#Count
pur_person_count=pur_person[["Price"]].count()
pur_person_count.head(50)
pur_person_count = pur_person_count.rename(columns = {"Price": "Purchase Count"}) 
person_stats = pd.merge(pur_person_ave, pur_person_count,how='outer', on="SN")

# Total purchase
pur_person_tot=pur_person[["Price"]].sum()
pur_person_tot = pur_person_tot.rename(columns = {"Price": "Total Purchase Value"}) 
person_stats = pd.merge(person_stats, pur_person_tot,how='outer', on="SN")

# Dataframe with stats
person_stats_sort=person_stats.sort_values(["Total Purchase Value"], axis=0, ascending=False)

#Formatting
person_stats_sort["Total Purchase Value"] = person_stats["Total Purchase Value"].astype(float).map("${:,.2f}".format)

person_stats_sort.head()



,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Lisosia93,$3.79,5,$18.96
Idastidru52,$3.86,4,$15.45
Chamjask73,$4.61,3,$13.83
Iral74,$3.40,4,$13.62
Iskadarya95,$4.37,3,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [545]:
#Group data by item ID and name
new_df=purchase_data[["Item ID", "Item Name", "Price"]]
new_df= new_df.groupby(["Item ID", "Item Name"])

# Purchase count
item_count=new_df[["Price"]].count()
item_count = item_count.rename(columns = {"Price": "Purchase Count"}) 
item_count.head()

# Item price
item_price=new_df[["Price"]].max()
item_price = item_price.rename(columns = {"Price": "Item Price"}) 
item_price["Item Price"] = item_price["Item Price"].astype(float).map("${:,.2f}".format)
item_stats = pd.merge(item_count, item_price,how='outer', on=['Item ID','Item Name'])
item_stats.head()

# Item total purchase
item_tot=new_df[["Price"]].sum()
item_tot = item_tot.rename(columns = {"Price": "Total Purchase"}) 
item_stats = pd.merge(item_stats, item_tot,how='outer', on=['Item ID','Item Name'])

# Dataframe with stats
item_stats_sort=item_stats.sort_values(["Purchase Count"], axis=0, ascending=False)
item_stats_sort["Total Purchase"] = item_stats_sort["Total Purchase"].astype(float).map("${:,.2f}".format)

item_stats_sort.head()

#item_stats_sort.min()


,,Purchase Count,Item Price,Total Purchase
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the data frame

In [547]:
# Sort by total purchase 
item_stats_pur=item_stats.sort_values("Total Purchase",axis=0,  ascending=False, na_position='last')
item_stats_pur["Total Purchase"] = item_stats_pur["Total Purchase"].astype(float).map("${:,.2f}".format)

item_stats_pur.head()

,,Purchase Count,Item Price,Total Purchase
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
